## Importando as bibliotecas e base de dados 

In [3]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

## Pré-processamento

### Carregando a base de dados

In [4]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

### Normalizando as imagens

Dividimos cada pixel das imagens das bases de treinamento e teste, utilizando o maior valor: 255.

Com isso, cada pixel estará na faixa entre 0 e 1. Dessa forma, a rede neural vai treinar mais rápida.

In [5]:
X_train = X_train / 255.0

In [6]:
X_test = X_test / 255.0

### Remodelando (reshaping)  a base de dados

Como estamos trabalhando com uma rede neural densa, mudamos a dimensão das bases de dados para ficarem no formato de vetor.

In [7]:
X_train.shape

(60000, 28, 28)

In [8]:
# Como a dimensão de cada imagem é 28x28, mudamos toda a base de dados para o formato [-1 (todos os elementos), largura * altura]
X_train = X_train.reshape(-1, 28 * 28)

In [9]:
X_train.shape

(60000, 784)

In [10]:
# Mudamos também a dimensão da base de teste
X_test = X_test.reshape(-1, 28*28)

## Construindo a Rede Neural Artificial

### Definindo o modelo modelo normal (não distribuido)

In [12]:
model_normal = tf.keras.models.Sequential()
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model_normal.add(tf.keras.layers.Dropout(rate=0.2))
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

## Definindo a estratégia distribuida

In [14]:
distribute = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


## Definindo um modelo distribuído

In [15]:
with distribute.scope():
    model_distributed = tf.keras.models.Sequential()
    model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
    model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
    model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
    model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Comparação de velocidade entre o treinamento normal e o treinamento distribuído

In [17]:
import time

starting_time = time.time()
model_distributed.fit(X_train, y_train, epochs=20, batch_size=128)
print("Distributed training: {}".format(time.time() - starting_time))

Epoch 1/20


2022-09-28 12:50:54.210115: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


469/469 [==============================] - 1s 1ms/step - loss: 0.5919 - sparse_categorical_accuracy: 0.7959
Epoch 2/20
469/469 [==============================] - 1s 1ms/step - loss: 0.4243 - sparse_categorical_accuracy: 0.8515
Epoch 3/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3825 - sparse_categorical_accuracy: 0.8641
Epoch 4/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3557 - sparse_categorical_accuracy: 0.8720
Epoch 5/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3406 - sparse_categorical_accuracy: 0.8773
Epoch 6/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3247 - sparse_categorical_accuracy: 0.8823
Epoch 7/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3129 - sparse_categorical_accuracy: 0.8862
Epoch 8/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3045 - sparse_categorical_accuracy: 0.8886
Epoch 9/20
469/469 [==============================] - 1s 1m

In [18]:
starting_time = time.time()
model_normal.fit(X_train, y_train, epochs=20, batch_size=128)
print("Normal training: {}".format(time.time() - starting_time))

Epoch 1/20
469/469 [==============================] - 1s 1ms/step - loss: 0.6038 - sparse_categorical_accuracy: 0.7886
Epoch 2/20
469/469 [==============================] - 1s 1ms/step - loss: 0.4319 - sparse_categorical_accuracy: 0.8478
Epoch 3/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3862 - sparse_categorical_accuracy: 0.8604
Epoch 4/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3604 - sparse_categorical_accuracy: 0.8701
Epoch 5/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3439 - sparse_categorical_accuracy: 0.8746
Epoch 6/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3299 - sparse_categorical_accuracy: 0.8799
Epoch 7/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3177 - sparse_categorical_accuracy: 0.8831
Epoch 8/20
469/469 [==============================] - 1s 1ms/step - loss: 0.3078 - sparse_categorical_accuracy: 0.8885
Epoch 9/20
469/469 [============================